<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250620_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Python基本操作

參考資料：https://hackmd.io/@OscarTAI/ryJFSbEuR

In [ ]:
!pip install google-cloud-bigquery

In [6]:
import os
from google.cloud import bigquery

# 設置 Google Cloud 認證
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/tibame-gad251-31-0613-c339499c7e0e.json"


# 初始化 BigQuery 客戶端
client = bigquery.Client()
print('connection done')


connection done


In [ ]:
#定義變數
dataset_id = 'tibame_gad251_31_dataset_py'  # 替換為想要建立的資料集名稱
project_id = 'tibame-gad251-31-0613'  # 替換為 Google Cloud 專案 ID
table_id = 'age_table'

In [ ]:
# 定義資料集參數

dataset_ref = client.dataset(dataset_id)


# 設定資料集的描述和其他選項
dataset = bigquery.Dataset(dataset_ref)
dataset.description = 'This is a new dataset created from Python.'
dataset.location = 'US'  # 設定資料集的地區位置

# 建立資料集
dataset = client.create_dataset(dataset, timeout=30)  # 可設定超時時間
print(f"dataset: {dataset.dataset_id} ,created")


In [ ]:
# 定義資料集和資料表
table_id = 'age_table'
schema = [
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("age", "INTEGER"),
]

# 定義表格參數
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)

# 建立資料表
table = client.create_table(table)
print(f"table: {table.table_id} ,created")


table: age_table ,created


In [ ]:
# 定義資料集和表資料表名稱
#dataset_id = '資料集ID'
#table_id = 'age_table'

# 定義資料
rows_to_insert = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 35},
    {"name":"Oscar","age": 29}
]

# 插入資料
errors = client.insert_rows_json(f"{dataset_id}.{table_id}", rows_to_insert)

if errors == []:
    print("data insert successfully")
else:
    print(f"error: {errors}")



data insert successfully


In [ ]:
# 定義查詢語句SQL
query = f"""
    SELECT name, age
    FROM `{project_id}.{dataset_id}.age_table`

"""

# 執行查詢
query_job = client.query(query)

# 獲取結果
results = query_job.result()

# 印出結果
for row in results:
    print(f"name: {row.name}, age: {row.age}")

print(results)

name: Alice, age: 25
name: Bob, age: 35
name: Oscar, age: 29


#### 建立模型

In [ ]:
# 定義 SQL 查詢以創建模型
query = f"""
CREATE OR REPLACE MODEL `{project_id}.{dataset_id}.imported_tf_model`
OPTIONS (MODEL_TYPE='TENSORFLOW',
         MODEL_PATH='gs://cloud-training-demos/txtclass/export/exporter/1549825580/*')
"""

# 執行 SQL 查詢以創建模型
query_job = client.query(query)
query_job.result()  # 等待查詢完成

print("TensorFlow model import BigQuery ML")


TensorFlow model import BigQuery ML


#### 使用模型

In [ ]:
# 定義 SQL 查詢以使用模型進行預測，只取五筆非 NULL 的資料
query = f"""
SELECT *
FROM ML.PREDICT(MODEL `{project_id}.{dataset_id}.imported_tf_model`,
  (SELECT title AS input
   FROM `bigquery-public-data.hacker_news.full`
   WHERE title IS NOT NULL
   LIMIT 5)  -- 限制結果為五筆資料
)
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

# 印出預測結果
for row in results:
    print(row)


Row(([0.062409449368715286, 0.28719037771224976, 0.6504001617431641], 'Top 7 Market and Product Research Survey Making Tools'), {'dense_1': 0, 'input': 1})
Row(([0.9210265874862671, 0.03631364926695824, 0.042659733444452286], 'How Often Should I Redesign My Website?'), {'dense_1': 0, 'input': 1})
Row(([0.0011751624988391995, 0.9958990216255188, 0.002925690496340394], 'JSON Parse Result Not Shown on Recyclerview?'), {'dense_1': 0, 'input': 1})
Row(([1.0, 9.858796736064285e-23, 1.6511799387627576e-14], 'BackboneJS with normal php server?'), {'dense_1': 0, 'input': 1})
Row(([0.9999982118606567, 1.777306871553891e-13, 1.7626597355047124e-06], 'What is new in bootstrap 4?'), {'dense_1': 0, 'input': 1})


In [ ]:
# 定義資料集參數
dataset_ref = client.dataset(dataset_id)

# 刪除資料集
client.cr
client.delete_dataset(dataset_ref, delete_contents=True)
print(f"dataset: {dataset_id} ,deleted")

### 線性回歸企鵝分析

In [7]:
us_dataset_id='tibame_gad251_31_dataset_us'#定義US資料集ID
eu_dataset_id='tibame_gad251_31_dataset_eu'#定義EU資料集ID

#### 建立線性迴歸模型

In [ ]:
# 定義 SQL 查詢以創建模型
query = f"""
CREATE OR REPLACE MODEL `{us_dataset_id}.penguins_model`
OPTIONS
  (model_type='linear_reg',
  input_label_cols=['body_mass_g']) AS
SELECT
  *
FROM
  `bigquery-public-data.ml_datasets.penguins`
WHERE
  body_mass_g IS NOT NULL;
"""

# 執行 SQL 查詢以創建模型
query_job = client.query(query)
query_job.result()  # 等待查詢完成

print("線性回歸 model import BigQuery ML")

線性回歸 model import BigQuery ML


#### 評估線性回歸模型

In [ ]:
# 定義 SQL 查詢以使用模型進行評估
query = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL `{us_dataset_id}.penguins_model`,
    (
    SELECT
      *
    FROM
      `bigquery-public-data.ml_datasets.penguins`
    WHERE
      body_mass_g IS NOT NULL));

"""

# 執行 SQL 查詢以進行評估
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [ ]:
# 取得 BigQuery 回傳的欄位名稱
field_names = results.schema

# 印出評估結果
for row in results:
    for field, value in zip(field_names, row):
        print(f"{field.name}: {value}")

mean_absolute_error: 227.0122366744722
mean_squared_error: 81838.15989216763
mean_squared_log_error: 0.0050704473735013016
median_absolute_error: 173.08081641661283
r2_score: 0.8723772534253442
explained_variance: 0.8723772534253442


#### 預測線性回歸模型

In [ ]:
# 定義 SQL 查詢以使用模型進行預測
query = f"""
SELECT
  *
FROM
  ML.PREDICT(MODEL `{us_dataset_id}.penguins_model`,
    (
    SELECT
      *
    FROM
      `bigquery-public-data.ml_datasets.penguins`
    WHERE island = 'Biscoe'));
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [ ]:
# 將每筆 row 解成 tuple（row.values() 是一整筆，row[i] 是值）
rows_data = [tuple(row.values()) for row in results]

# 取得欄位名稱
columns = [field.name for field in results.schema]

# 建立 DataFrame
import pandas as pd
df = pd.DataFrame(rows_data, columns=columns)

# 顯示前幾筆資料
print(df.head())


   predicted_body_mass_g                              species  island  \
0            4681.782896    Gentoo penguin (Pygoscelis papua)  Biscoe   
1            3875.224470  Adelie Penguin (Pygoscelis adeliae)  Biscoe   
2            3303.096891  Adelie Penguin (Pygoscelis adeliae)  Biscoe   
3            3976.529009  Adelie Penguin (Pygoscelis adeliae)  Biscoe   
4            3457.923587  Adelie Penguin (Pygoscelis adeliae)  Biscoe   

   culmen_length_mm  culmen_depth_mm  flipper_length_mm  body_mass_g     sex  
0               NaN              NaN                NaN          NaN    None  
1              39.7             18.9              184.0       3550.0    MALE  
2              36.4             17.1              184.0       2850.0  FEMALE  
3              41.6             18.0              192.0       3950.0    MALE  
4              35.0             17.9              192.0       3725.0  FEMALE  


#### 使用預測解釋線性回歸模型

In [ ]:
# 定義 SQL 查詢以使用模型進行預測解釋
query = f"""
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL `{us_dataset_id}.penguins_model`,
    (
    SELECT
      *
    FROM
      `bigquery-public-data.ml_datasets.penguins`
    WHERE island = 'Biscoe'),
    STRUCT(3 as top_k_features));
"""

# 執行 SQL 查詢以進行預測解釋
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [ ]:
# 將每筆 row 解成 tuple（row.values() 是一整筆，row[i] 是值）
rows_data = [tuple(row.values()) for row in results]

# 取得欄位名稱
columns = [field.name for field in results.schema]

# 建立 DataFrame
import pandas as pd
df = pd.DataFrame(rows_data, columns=columns)

# 顯示前幾筆資料
print(df.head())

   predicted_body_mass_g                           top_feature_attributions  \
0            4681.782896  [{'feature': 'island', 'attribution': 2483.386...   
1            3875.224470  [{'feature': 'island', 'attribution': 2483.386...   
2            3303.096891  [{'feature': 'island', 'attribution': 2483.386...   
3            3976.529009  [{'feature': 'island', 'attribution': 2483.386...   
4            3457.923587  [{'feature': 'island', 'attribution': 2483.386...   

   baseline_prediction_value  prediction_value  approximation_error  \
0                 609.307581       4681.782896                  0.0   
1                 609.307581       3875.224470                  0.0   
2                 609.307581       3303.096891                  0.0   
3                 609.307581       3976.529009                  0.0   
4                 609.307581       3457.923587                  0.0   

                               species  island  culmen_length_mm  \
0    Gentoo penguin (Pygosceli

In [ ]:
df = df[['top_feature_attributions', 'baseline_prediction_value', 'prediction_value']]
pd.set_option('display.max_colwidth', None)#取消自動截斷
print(df.head())


                                                                                                                                                       top_feature_attributions  \
0  [{'feature': 'island', 'attribution': 2483.3867091343363}, {'feature': 'species', 'attribution': 1258.2299000338294}, {'feature': 'sex', 'attribution': 330.85870621071217}]   
1    [{'feature': 'island', 'attribution': 2483.3867091343363}, {'feature': 'sex', 'attribution': 740.4273092703097}, {'feature': 'species', 'attribution': 271.2537788676088}]   
2    [{'feature': 'island', 'attribution': 2483.3867091343363}, {'feature': 'sex', 'attribution': 361.1858329646648}, {'feature': 'species', 'attribution': 271.2537788676088}]   
3    [{'feature': 'island', 'attribution': 2483.3867091343363}, {'feature': 'sex', 'attribution': 740.4273092703097}, {'feature': 'species', 'attribution': 271.2537788676088}]   
4    [{'feature': 'island', 'attribution': 2483.3867091343363}, {'feature': 'sex', 'attribution': 361.185

In [ ]:
# 將每列的 attribution 陣列拆開（每個特徵貢獻變一列）
df_exploded = df.explode('top_feature_attributions').reset_index(drop=True)

# 展開 struct 欄位（如 {'feature': 'island', 'attribution': 2483.38}）
attr = pd.json_normalize(df_exploded['top_feature_attributions'])

# 合併 baseline、prediction 與 attribution 欄位
df_result = pd.concat([
    df_exploded[['baseline_prediction_value', 'prediction_value']],
    attr
], axis=1)

# 顯示結果
print(df_result.head(10))


   baseline_prediction_value  prediction_value  feature  attribution
0                 609.307581       4681.782896   island  2483.386709
1                 609.307581       4681.782896  species  1258.229900
2                 609.307581       4681.782896      sex   330.858706
3                 609.307581       3875.224470   island  2483.386709
4                 609.307581       3875.224470      sex   740.427309
5                 609.307581       3875.224470  species   271.253779
6                 609.307581       3303.096891   island  2483.386709
7                 609.307581       3303.096891      sex   361.185833
8                 609.307581       3303.096891  species   271.253779
9                 609.307581       3976.529009   island  2483.386709


#### 重新訓練可全局解釋線性回歸模型

In [ ]:
# 定義 SQL 查詢以重新訓練可全局解釋模型
query = f"""
CREATE OR REPLACE MODEL `{us_dataset_id}.penguins_model`
  OPTIONS (
    model_type = 'linear_reg',
    input_label_cols = ['body_mass_g'],
    enable_global_explain = TRUE)
AS
SELECT
  *
FROM
  `bigquery-public-data.ml_datasets.penguins`
WHERE
  body_mass_g IS NOT NULL;

"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成
print('已建立全局解釋線性回歸模型')

已建立全局解釋線性回歸模型


#### 執行全局解釋

In [3]:
# 定義 SQL 查詢以使用模型進行預測
query = f"""
SELECT
  *
FROM
  ML.GLOBAL_EXPLAIN(MODEL `{us_dataset_id}.penguins_model`);
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [5]:
# 將每筆 row 解成 tuple（row.values() 是一整筆，row[i] 是值）
rows_data = [tuple(row.values()) for row in results]

# 取得欄位名稱
columns = [field.name for field in results.schema]

# 建立 DataFrame
import pandas as pd
df = pd.DataFrame(rows_data, columns=columns)

# 顯示前幾筆資料
print(df.head())

             feature   attribution
0             island  14725.672885
1                sex   5305.149684
2            species    816.564124
3  flipper_length_mm    193.612051
4    culmen_depth_mm    117.084944


### 分類模型人口普查分析

#### 準本樣本資料

In [8]:
# 定義 SQL 查詢以準備樣本資料檢視表
query = f"""
CREATE OR REPLACE VIEW
`{us_dataset_id}.input_data` AS
SELECT
  age,
  workclass,
  marital_status,
  education_num,
  occupation,
  hours_per_week,
  income_bracket,
  CASE
    WHEN MOD(functional_weight, 10) < 8 THEN 'training'
    WHEN MOD(functional_weight, 10) = 8 THEN 'evaluation'
    WHEN MOD(functional_weight, 10) = 9 THEN 'prediction'
  END AS dataframe
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`

"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

#### 建立分類模型

In [9]:
# 定義 SQL 查詢以建立模型並送入檢視表的訓練資料
query = f"""
CREATE OR REPLACE MODEL
`{us_dataset_id}.census_model`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    data_split_method='NO_SPLIT',
    input_label_cols=['income_bracket'],
    max_iterations=15) AS
SELECT * EXCEPT(dataframe)
FROM
  `{us_dataset_id}.input_data`
WHERE
  dataframe = 'training'

"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

#### 利用評估資料模型評估

In [10]:
# 定義 SQL 查詢以使用模型進行預測
query = f"""
SELECT
  *
FROM
  ML.EVALUATE (MODEL `{us_dataset_id}.census_model`,
    (
    SELECT
      age,
      workclass,
      marital_status,
      education_num,
      occupation,
      hours_per_week,
      income_bracket,
    FROM
      `{us_dataset_id}.input_data`
    WHERE
      dataframe = 'evaluation'
    )
  )
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [11]:
# 將每筆 row 解成 tuple（row.values() 是一整筆，row[i] 是值）
rows_data = [tuple(row.values()) for row in results]

# 取得欄位名稱
columns = [field.name for field in results.schema]

# 建立 DataFrame
import pandas as pd
df = pd.DataFrame(rows_data, columns=columns)

# 顯示前幾筆資料
print(df.head())

   precision    recall  accuracy  f1_score  log_loss   roc_auc
0   0.574055  0.763587  0.816801  0.655394  0.394407  0.882743


#### 利用預測資料進行模型預測

In [12]:
# 使用ML.PREDICT函數預測特定受訪者的收入範圍
query = f"""
SELECT
  *
FROM
  ML.PREDICT (MODEL `{us_dataset_id}.census_model`,
    (
    SELECT
      age,
      workclass,
      marital_status,
      education_num,
      occupation,
      hours_per_week,
      income_bracket,
    FROM
      `{us_dataset_id}.input_data`
    WHERE
      dataframe = 'prediction'
    )
  )
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [13]:
# 將每筆 row 解成 tuple（row.values() 是一整筆，row[i] 是值）
rows_data = [tuple(row.values()) for row in results]

# 取得欄位名稱
columns = [field.name for field in results.schema]

# 建立 DataFrame
import pandas as pd
df = pd.DataFrame(rows_data, columns=columns)

# 顯示前幾筆資料
print(df.head())

  predicted_income_bracket                     predicted_income_bracket_probs  \
0                    <=50K  [{'label': ' >50K', 'prob': 0.1147954679143910...   
1                    <=50K  [{'label': ' >50K', 'prob': 0.1071632184174308...   
2                    <=50K  [{'label': ' >50K', 'prob': 0.2839871714483840...   
3                    <=50K  [{'label': ' >50K', 'prob': 0.2318287733959611...   
4                    <=50K  [{'label': ' >50K', 'prob': 0.1096615166295337...   

   age   workclass       marital_status  education_num      occupation  \
0   52   Local-gov   Married-civ-spouse              5   Other-service   
1   75           ?   Married-civ-spouse              5               ?   
2   74           ?   Married-civ-spouse              5               ?   
3   66           ?   Married-civ-spouse              5               ?   
4   45     Private   Married-civ-spouse              5   Other-service   

   hours_per_week income_bracket  
0              40          <=50K 

In [14]:
pd.set_option('display.max_colwidth', None) #取消自動截斷
# 顯示前幾筆資料
print(df.head())

  predicted_income_bracket  \
0                    <=50K   
1                    <=50K   
2                    <=50K   
3                    <=50K   
4                    <=50K   

                                                                       predicted_income_bracket_probs  \
0  [{'label': ' >50K', 'prob': 0.11479546791439109}, {'label': ' <=50K', 'prob': 0.8852045320856089}]   
1  [{'label': ' >50K', 'prob': 0.10716321841743089}, {'label': ' <=50K', 'prob': 0.8928367815825691}]   
2   [{'label': ' >50K', 'prob': 0.28398717144838403}, {'label': ' <=50K', 'prob': 0.716012828551616}]   
3   [{'label': ' >50K', 'prob': 0.2318287733959611}, {'label': ' <=50K', 'prob': 0.7681712266040389}]   
4  [{'label': ' >50K', 'prob': 0.10966151662953375}, {'label': ' <=50K', 'prob': 0.8903384833704663}]   

   age   workclass       marital_status  education_num      occupation  \
0   52   Local-gov   Married-civ-spouse              5   Other-service   
1   75           ?   Married-civ-spouse 

#### 解釋預測結果

In [15]:
# 定義 SQL 查詢以使用模型進行預測
query = f"""
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL `{us_dataset_id}.census_model`,
    (
    SELECT
      age,
      workclass,
      marital_status,
      education_num,
      occupation,
      hours_per_week,
      income_bracket,
    FROM
      `{us_dataset_id}.input_data`
    WHERE
      dataframe = 'prediction'),
    STRUCT(3 as top_k_features))

"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [16]:
# 將每筆 row 解成 tuple（row.values() 是一整筆，row[i] 是值）
rows_data = [tuple(row.values()) for row in results]

# 取得欄位名稱
columns = [field.name for field in results.schema]

# 建立 DataFrame
import pandas as pd
df = pd.DataFrame(rows_data, columns=columns)
pd.set_option('display.max_colwidth', None) #取消自動截斷
# 顯示前幾筆資料
print(df.head())

  predicted_income_bracket  probability  \
0                    <=50K     0.885205   
1                    <=50K     0.892837   
2                    <=50K     0.716013   
3                    <=50K     0.768171   
4                    <=50K     0.890338   

                                                                                                                                                                             top_feature_attributions  \
0  [{'feature': 'education_num', 'attribution': -1.5192713075870252}, {'feature': 'occupation', 'attribution': -1.1894058413766737}, {'feature': 'marital_status', 'attribution': 0.954697173569993}]   
1         [{'feature': 'education_num', 'attribution': -1.5192713075870252}, {'feature': 'hours_per_week', 'attribution': -1.2457165769417757}, {'feature': 'age', 'attribution': 1.244208014372101}]   
2           [{'feature': 'education_num', 'attribution': -1.5192713075870252}, {'feature': 'age', 'attribution': 1.210052440803534}, {'fea

#### 重新訓練可以全局解釋的模型

In [17]:
# 定義 SQL 查詢以建立模型並送入檢視表的訓練資料
query = f"""
CREATE OR REPLACE MODEL
`{us_dataset_id}.census_model`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    data_split_method='NO_SPLIT',
    enable_global_explain=TRUE,
    input_label_cols=['income_bracket'],
    max_iterations=15) AS
SELECT * EXCEPT(dataframe)
FROM
  `{us_dataset_id}.input_data`
WHERE
  dataframe = 'training'
"""

# 執行 SQL 查詢
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [19]:
# 定義 SQL 查詢以使用模型進行預測
query = f"""
SELECT
  *
FROM
  ML.GLOBAL_EXPLAIN(MODEL `{us_dataset_id}.census_model`);
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [20]:
# 將每筆 row 解成 tuple（row.values() 是一整筆，row[i] 是值）
rows_data = [tuple(row.values()) for row in results]

# 取得欄位名稱
columns = [field.name for field in results.schema]

# 建立 DataFrame
import pandas as pd
df = pd.DataFrame(rows_data, columns=columns)

#取消自動截斷
pd.set_option('display.max_colwidth', None)
# 顯示前幾筆資料
print(df.head())

          feature  attribution
0  marital_status     1.178824
1   education_num     0.569518
2      occupation     0.520015
3             age     0.381932
4       workclass     0.310469
